# Single Model Training

This notebook is to enable model training for a single model using the specified parameters in config.yaml.


In [1]:
# Import required libraries
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..", "..",))
sys.path.append(project_root)
from src.pipelines.xgboost.train_single_model import main

## 1. Run Model

In [ ]:
features = ["fea_is_foods", "fea_is_household", "fea_is_hobbies", "fea_dept_number"]
trained_model = main()

## 2. Compare metrics with Benchmark

1. Compare Metrics (MdAPE, MdAE) with benchmark in total and accross validation sets of interest

In [ ]:
# Now you can import from core if needed
from src.core.clients.bigquery import BigQueryClient

bq_client = BigQueryClient()

sales_query = f"""
    SELECT 
        tgt.ctx_date_month,
        tgt.ctx_cat_id, 
        tgt.ctx_dept_id, 
        tgt.ctx_item_id,
        tgt.tgt_benchmark,
        tgt.tgt_monthly_sales_sum_3_next_months,
        {','.join(features)}
    FROM `porygon-pipelines.walmart_training_tables.walmart_master_table` tgt
    WHERE 
        tgt.ctx_store_id = 'CA_1'
        AND is_stockout_tgt = 0
"""


# Extract data
df = bq_client.load_from_query(sales_query)


INFO:src.core.clients.bigquery:BigQuery client initialized for project: porygon-pipelines
/Users/anapreciado/Desktop/porygon-demand-forecasting/venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
INFO:src.core.clients.bigquery:Loaded 148964 rows from custom query.


In [ ]:
from src.core.transformations.transformations import split_dataset

# Split dataframes
training_df, validation_df = split_dataset(loaded_df= df)

## 3. Evaluate Biass

## 4. Evaluate Variance

## 5. Evaluate Biass-Variance Trade Off

## 6. Evaluate Residuals

## 7. Error Distribution

## 8. Feature Importance

## 9. Feature Drift

## 10. SHAP Values